# <span style="font-family:Courier New; color:#CCCCCC">**Matriu de Dades processades**</span>

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import pylab
import math
import scipy.stats as stats
from sklearn.covariance import MinCovDet

warnings.filterwarnings("ignore")

In [33]:
daily = pd.read_csv("../data/preprocessed/states_daily_cleaned.csv")
vac = pd.read_csv("../data/preprocessed/vacc_clean.csv")

In [34]:
daily

,date,state,positive,totalTestResults,death,positiveIncrease,negativeIncrease,total,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease
0,2020-03-06,AK,0.0,8.0,0.0,0,0,1,0,0,0,0
1,2020-03-07,AK,0.0,12.0,0.0,0,0,2,4,0,0,0
2,2020-03-08,AK,0.0,14.0,0.0,0,0,6,2,0,0,0
3,2020-03-09,AK,0.0,23.0,0.0,0,0,9,9,0,0,1
4,2020-03-10,AK,0.0,23.0,0.0,0,0,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20775,2021-03-03,WY,54616.0,643476.0,682.0,89,278,236297,2096,236297,0,3
20776,2021-03-04,WY,54685.0,646179.0,682.0,69,276,236642,2703,236642,0,1
20777,2021-03-05,WY,54764.0,649293.0,682.0,79,307,237028,3114,237028,0,2
20778,2021-03-06,WY,54764.0,649293.0,682.0,0,0,237028,0,237028,0,0


In [35]:
drop_states = ['GU', 'AS', 'MP', 'VI', 'PR']
daily = daily[~daily['state'].isin(drop_states)]

us_state_abbrev = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
    'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana',
    'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota',
    'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
    'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia', 'PR': 'Puerto Rico'
    }

daily['state'] = daily['state'].map(us_state_abbrev).fillna(daily['state'])


In [36]:
vac = vac.rename(columns={'State': 'state', 'Date': 'date'})

In [37]:
vac

,state,date,Dose1_Total,Dose1_Total_pct,Dose1_65Plus,Dose1_65Plus_pct,Complete_Total,Complete_Total_pct,Complete_65Plus,Complete_65Plus_pct,Low_SVI_CTGY,Moderate_Low_SVI_CTGY,Moderate_High_SVI_CTGY,High_SVI_CTGY,Metro,Non-metro,Census2019,Census2019_65Plus
0,Alabama,2021-01-07,23162.0,0.472387,2857.0,0.334421,1244.0,0.025371,134.0,0.015685,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
1,Alabama,2021-01-08,23369.0,0.476609,2958.0,0.346244,1375.0,0.028043,142.0,0.016622,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
2,Alabama,2021-01-09,42440.0,0.865560,5687.0,0.665682,5341.0,0.108929,484.0,0.056654,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
3,Alabama,2021-01-10,43359.0,0.884303,5903.0,0.690965,5903.0,0.120391,579.0,0.067774,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
4,Alabama,2021-01-11,43427.0,0.885690,5936.0,0.694828,5908.0,0.120493,579.0,0.067774,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3064,Wyoming,2021-03-03,100955.0,17.443357,56897.0,57.594469,56558.0,9.772289,33069.0,33.474375,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
3065,Wyoming,2021-03-04,102374.0,17.688537,57395.0,58.098574,58499.0,10.107661,34292.0,34.712367,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
3066,Wyoming,2021-03-05,107956.0,18.653014,58883.0,59.604814,63585.0,10.986438,37223.0,37.679296,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
3067,Wyoming,2021-03-06,108266.0,18.706577,58947.0,59.669599,63869.0,11.035509,37313.0,37.770400,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0


In [38]:
# Fem el merge dels dos datasets
merged = pd.merge(daily, vac, on=['state', 'date'], how='outer') 
merged

,date,state,positive,totalTestResults,death,positiveIncrease,negativeIncrease,total,totalTestResultsIncrease,posNeg,...,Complete_65Plus,Complete_65Plus_pct,Low_SVI_CTGY,Moderate_Low_SVI_CTGY,Moderate_High_SVI_CTGY,High_SVI_CTGY,Metro,Non-metro,Census2019,Census2019_65Plus
0,2020-03-06,Alaska,0.0,8.0,0.0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-07,Alaska,0.0,12.0,0.0,0,0,2,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-08,Alaska,0.0,14.0,0.0,0,0,6,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-09,Alaska,0.0,23.0,0.0,0,0,9,9,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-10,Alaska,0.0,23.0,0.0,0,0,9,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,2021-03-03,Wyoming,54616.0,643476.0,682.0,89,278,236297,2096,236297,...,33069.0,33.474375,33.58,57.08,9.34,0.0,30.99,69.01,578759.0,98789.0
18991,2021-03-04,Wyoming,54685.0,646179.0,682.0,69,276,236642,2703,236642,...,34292.0,34.712367,33.58,57.08,9.34,0.0,30.99,69.01,578759.0,98789.0
18992,2021-03-05,Wyoming,54764.0,649293.0,682.0,79,307,237028,3114,237028,...,37223.0,37.679296,33.58,57.08,9.34,0.0,30.99,69.01,578759.0,98789.0
18993,2021-03-06,Wyoming,54764.0,649293.0,682.0,0,0,237028,0,237028,...,37313.0,37.770400,33.58,57.08,9.34,0.0,30.99,69.01,578759.0,98789.0


In [39]:
print(merged.isna().sum()[merged.isna().sum() > 0])

Dose1_Total               15926
Dose1_Total_pct           15926
Dose1_65Plus              15926
Dose1_65Plus_pct          15926
Complete_Total            15926
Complete_Total_pct        15926
Complete_65Plus           15926
Complete_65Plus_pct       15926
Low_SVI_CTGY              15926
Moderate_Low_SVI_CTGY     15926
Moderate_High_SVI_CTGY    15926
High_SVI_CTGY             15926
Metro                     15926
Non-metro                 15926
Census2019                15926
Census2019_65Plus         15926
dtype: int64


In [40]:
cols_to_impute = [
    'Low_SVI_CTGY', 'Moderate_Low_SVI_CTGY', 'Moderate_High_SVI_CTGY', 'High_SVI_CTGY',
    'Metro', 'Non-metro', 'Census2019', 'Census2019_65Plus'
]

for col in cols_to_impute:
    merged[col] = merged.groupby('state')[col].transform(
        lambda x: x.fillna(x.dropna().iloc[0]) if x.dropna().any() else x
    )

for col in cols_to_impute:
    merged[col] = merged.apply(
        lambda row: row[col] if row[cols_to_impute].isna().all() else row[col] if pd.notna(row[col]) else 0, axis=1
    )

    


In [41]:
cols_to_fill = [
    'Dose1_Total', 'Dose1_Total_pct', 'Dose1_65Plus', 'Dose1_65Plus_pct',
    'Complete_Total', 'Complete_Total_pct', 'Complete_65Plus', 'Complete_65Plus_pct'
]

merged[cols_to_fill] = merged[cols_to_fill].fillna(0)

In [42]:
merged = merged.sort_values(by=['state', 'date']).reset_index(drop=True)
merged

,date,state,positive,totalTestResults,death,positiveIncrease,negativeIncrease,total,totalTestResultsIncrease,posNeg,...,Complete_65Plus,Complete_65Plus_pct,Low_SVI_CTGY,Moderate_Low_SVI_CTGY,Moderate_High_SVI_CTGY,High_SVI_CTGY,Metro,Non-metro,Census2019,Census2019_65Plus
0,2020-03-07,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
1,2020-03-08,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
2,2020-03-09,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
3,2020-03-10,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
4,2020-03-11,Alabama,0.0,10.0,0.0,0,10,10,10,10,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,2021-03-03,Wyoming,54616.0,643476.0,682.0,89,278,236297,2096,236297,...,33069.0,33.474375,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
18991,2021-03-04,Wyoming,54685.0,646179.0,682.0,69,276,236642,2703,236642,...,34292.0,34.712367,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
18992,2021-03-05,Wyoming,54764.0,649293.0,682.0,79,307,237028,3114,237028,...,37223.0,37.679296,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
18993,2021-03-06,Wyoming,54764.0,649293.0,682.0,0,0,237028,0,237028,...,37313.0,37.770400,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0


In [43]:
print(merged.isna().sum()[merged.isna().sum() > 0])


Low_SVI_CTGY              739
Moderate_Low_SVI_CTGY     739
Moderate_High_SVI_CTGY    739
High_SVI_CTGY             739
Metro                     739
Non-metro                 739
Census2019                739
Census2019_65Plus         739
dtype: int64


In [44]:
df = pd.read_csv("../data/raw/Vacunació/COVID-19_Vaccinations_in_the_United_States_County.csv")
df = df[df['Recip_State'] == 'TX']
df = df[['Recip_State','Recip_County', 'SVI_CTGY', 'Census2019', 'Metro_status']]
df['Census2019'] = pd.to_numeric(df['Census2019'], errors='coerce').fillna(0).astype(float)

mapping = {
    "A": "Low", # (0.0, 0.25): "Low",
    "B": "Moderate-Low", # (0.25, 0.5): "Moderate-Low",
    "C": "Moderate-High", # (0.5, 0.75): "Moderate-High",
    "D": "High" # (0.75, 1.0): "High"
}

df["SVI_CTGY"] = df["SVI_CTGY"].replace(mapping)
df['SVI_CTGY'] = pd.Categorical(df['SVI_CTGY'], categories=mapping.values(), ordered=True)

df_counts_svi = df.drop_duplicates(subset=['Recip_State', 'Recip_County']).groupby(['Recip_State','SVI_CTGY'])['Census2019'].sum().reset_index().pivot(index='Recip_State', columns='SVI_CTGY', values='Census2019').fillna(0)
df_counts_svi = (df_counts_svi.div(df_counts_svi.sum(axis=1), axis=0) * 100).round(2)

column_rename_map = {
    'Low': 'Low_SVI_CTGY',
    'Moderate-Low': 'Moderate_Low_SVI_CTGY',
    'Moderate-High': 'Moderate_High_SVI_CTGY',
    'High': 'High_SVI_CTGY'
}
df_counts_svi = df_counts_svi.rename(columns=column_rename_map)

df_counts_metro = df.drop_duplicates(subset=['Recip_State', 'Recip_County']).groupby(['Recip_State','Metro_status'])['Census2019'].sum().reset_index().pivot(index='Recip_State', columns='Metro_status', values='Census2019').fillna(0)
df_counts_metro = (df_counts_metro.div(df_counts_metro.sum(axis=1), axis=0) * 100).round(2)

texas_df = merged[merged['state'] == 'Texas']

cols_to_impute = [
    'Low_SVI_CTGY', 'Moderate_Low_SVI_CTGY', 'Moderate_High_SVI_CTGY',
    'High_SVI_CTGY', 'Metro', 'Non-metro'
]

for col in cols_to_impute:
    if col in df_counts_svi.columns: 
        texas_df[col] = texas_df[col].fillna(df_counts_svi[col].iloc[0])  
    elif col in df_counts_metro.columns: 
        texas_df[col] = texas_df[col].fillna(df_counts_metro[col].iloc[0])  
texas_df['Census2019'] = texas_df['Census2019'].fillna(28986794)
texas_df['Census2019_65Plus'] = texas_df['Census2019_65Plus'].fillna(3738727)

merged.update(texas_df)


In [45]:
df = pd.read_csv("../data/raw/Vacunació/COVID-19_Vaccinations_in_the_United_States_County.csv")
df = df[df['Recip_State'] == 'HI']
df = df[['Recip_State','Recip_County', 'SVI_CTGY', 'Census2019', 'Metro_status']]
df['Census2019'] = pd.to_numeric(df['Census2019'], errors='coerce').fillna(0).astype(float)
mapping = {
    "A": "Low", # (0.0, 0.25): "Low",
    "B": "Moderate-Low", # (0.25, 0.5): "Moderate-Low",
    "C": "Moderate-High", # (0.5, 0.75): "Moderate-High",
    "D": "High" # (0.75, 1.0): "High"
}

df["SVI_CTGY"] = df["SVI_CTGY"].replace(mapping)
df['SVI_CTGY'] = pd.Categorical(df['SVI_CTGY'], categories=mapping.values(), ordered=True)

df_counts_svi = df.drop_duplicates(subset=['Recip_State', 'Recip_County']).groupby(['Recip_State','SVI_CTGY'])['Census2019'].sum().reset_index().pivot(index='Recip_State', columns='SVI_CTGY', values='Census2019').fillna(0)
df_counts_svi = (df_counts_svi.div(df_counts_svi.sum(axis=1), axis=0) * 100).round(2)

column_rename_map = {
    'Low': 'Low_SVI_CTGY',
    'Moderate-Low': 'Moderate_Low_SVI_CTGY',
    'Moderate-High': 'Moderate_High_SVI_CTGY',
    'High': 'High_SVI_CTGY'
}
df_counts_svi = df_counts_svi.rename(columns=column_rename_map)

df_counts_metro = df.drop_duplicates(subset=['Recip_State', 'Recip_County']).groupby(['Recip_State','Metro_status'])['Census2019'].sum().reset_index().pivot(index='Recip_State', columns='Metro_status', values='Census2019').fillna(0)
df_counts_metro = (df_counts_metro.div(df_counts_metro.sum(axis=1), axis=0) * 100).round(2)

hawaii_df = merged[merged['state'] == 'Hawaii']

cols_to_impute = [
    'Low_SVI_CTGY', 'Moderate_Low_SVI_CTGY', 'Moderate_High_SVI_CTGY',
    'High_SVI_CTGY', 'Metro', 'Non-metro'
]

for col in cols_to_impute:
    if col in df_counts_svi.columns:  
        hawaii_df[col] = hawaii_df[col].fillna(df_counts_svi[col].iloc[0])
    elif col in df_counts_metro.columns:
        hawaii_df[col] = hawaii_df[col].fillna(df_counts_metro[col].iloc[0]) 

hawaii_df['Census2019'] = hawaii_df['Census2019'].fillna(28986794)
hawaii_df['Census2019_65Plus'] = hawaii_df['Census2019_65Plus'].fillna(3738727)

merged.update(hawaii_df)


In [46]:
print(merged.isna().sum()[merged.isna().sum() > 0])

Series([], dtype: int64)


In [47]:
merged

,date,state,positive,totalTestResults,death,positiveIncrease,negativeIncrease,total,totalTestResultsIncrease,posNeg,...,Complete_65Plus,Complete_65Plus_pct,Low_SVI_CTGY,Moderate_Low_SVI_CTGY,Moderate_High_SVI_CTGY,High_SVI_CTGY,Metro,Non-metro,Census2019,Census2019_65Plus
0,2020-03-07,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
1,2020-03-08,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
2,2020-03-09,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
3,2020-03-10,Alabama,0.0,0.0,0.0,0,0,0,0,0,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
4,2020-03-11,Alabama,0.0,10.0,0.0,0,10,10,10,10,...,0.0,0.000000,8.99,15.97,43.99,31.04,76.84,23.16,4903185.0,854312.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,2021-03-03,Wyoming,54616.0,643476.0,682.0,89,278,236297,2096,236297,...,33069.0,33.474375,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
18991,2021-03-04,Wyoming,54685.0,646179.0,682.0,69,276,236642,2703,236642,...,34292.0,34.712367,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
18992,2021-03-05,Wyoming,54764.0,649293.0,682.0,79,307,237028,3114,237028,...,37223.0,37.679296,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0
18993,2021-03-06,Wyoming,54764.0,649293.0,682.0,0,0,237028,0,237028,...,37313.0,37.770400,33.58,57.08,9.34,0.00,30.99,69.01,578759.0,98789.0


Crear la nova columna que ens donarà informació dels estats veïns

In [51]:
neighbors = pd.read_csv("../data/raw/neighbor states/neighbor_states.csv")

In [52]:
def get_neighbors_contagions(state, date, merged, neighbors):
    neighbor_states = neighbors.loc[neighbors['state'] == state].drop('state', axis=1)
    
    neighbor_states = neighbor_states.columns[neighbor_states.iloc[0] == 1].tolist()
      
    neighbor_data = merged[(merged['state'].isin(neighbor_states)) & (merged['date'] == date)]
    return neighbor_data['positiveIncrease'].sum()

merged['neighbor_contagions'] = merged.apply(
    lambda row: get_neighbors_contagions(row['state'], row['date'], merged, neighbors), axis=1)


In [58]:
static = ["state", "Low_SVI_CTGY", "Moderate_Low_SVI_CTGY", "Moderate_High_SVI_CTGY", "High_SVI_CTGY", "Metro", "Non-metro", "Census2019", "Census2019_65Plus"]
static_df = merged[static].drop_duplicates()
static.remove("state")

merged = merged.drop(columns=static)

In [60]:
merged.to_csv("daily_covidMatrix.csv", index=False)
static_df.to_csv("static_stateMatrix.csv", index=False)